In [ ]:
import os
import shutil
import fastkaggle
from pathlib import Path
import numpy as np
import pandas as pd
import polars as pl
from datasets import Dataset
import torch  # base
import torch.nn.functional as F
import json
from transformers import (
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    AutoTokenizer,
    DataCollatorWithPadding,
)

In [ ]:
model_name = "answerdotai/ModernBERT-base"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Some weights of ModernBertForSequenceClassification were not initialized from the model checkpoint at answerdotai/ModernBERT-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
data_base_path = Path("./data")
comp_name = "llm-classification-finetuning"
datapath = data_base_path / comp_name
if not os.path.exists(datapath) and not datapath.exists():
    install_path = fastkaggle.setup_comp(comp_name)
    shutil.move(install_path, datapath)

In [ ]:
pl.Config.set_tbl_width_chars(300)
pl.Config.set_fmt_str_lengths(300)

polars.config.Config

In [ ]:
df_sample = pl.read_csv(datapath / "sample_submission.csv")
df_sample.head()

id,winner_model_a,winner_model_b,winner_tie
i64,f64,f64,f64
136060,0.333333,0.333333,0.333333
211333,0.333333,0.333333,0.333333
1233961,0.333333,0.333333,0.333333


In [ ]:
df_train = pl.read_csv(datapath / "train.csv")
df_test = pl.read_csv(datapath / "test.csv")
df_train.head()

FileNotFoundError: No such file or directory (os error 2): data/llm-classification-finetuning/train.csv

In [ ]:
def preprocess_function(batch, column):
    tokenizer(batch[column], truncation=True, padding=True)

In [ ]:
Dataset.from_pandas(df_test.to_pandas())

Dataset({
    features: ['id', 'prompt', 'response_a', 'response_b'],
    num_rows: 3
})

In [ ]:
df_test.dtypes

[Int64, String, String, String]

In [ ]:
cols = ["prompt", "response_a", "response_b"]
decoded = df_test.with_columns(
    [pl.col(col).str.json_decode() for col in cols if col in df_test.columns]
)
decoded.dtypes

[Int64, List(String), List(String), List(String)]